In [47]:
import pandas as pd
import time
from selenium import webdriver

PATH = "C:\WebDriver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

natixis_link = "https://recrutement.natixis.com/fr/jobs/list?contractType=NA_INTERNSHIP&country=FR"

irrelevant_info = ['Référence : ','Date de publication : ','Entité : ','Pays : ','Ville : ','Type de contrat : ']

In [48]:
#Ouverture du site et acceptation des cookies
driver.get(natixis_link)
if driver.find_element_by_id("cookieContents"):
    driver.find_element_by_xpath('''//*[@id="cookieContents"]/div/div/span[3]''').click()
time.sleep(1)
driver.get(natixis_link)

In [49]:
#Comptage du nombre de pages
page = 1
page_number = driver.find_element_by_class_name("pagination_custom_active").text
page_number = int(page_number.replace("{}/".format(page),""))

In [50]:
#Ici nous cherchons à savoir quel est l'id de la dernière offre qui a été enregistrée
last_csv = pd.read_csv("natixis.csv", index_col=[0])
last_csv_id = last_csv.iat[0,0]
last_csv_id

'4890'

In [51]:
#Ici on obtient le texte des offres. Ensuite, on nettoie les informations qui ne sont pas importantes en tenant compte du fait qu'il n'y aura pas toujours dix offres par page.
texte = driver.find_element_by_id("job_list_results").text
texte = texte.splitlines()
list_of_positions = texte.copy()
x = 1
nb_max_result = (len(texte)/2)+1
while x < nb_max_result:
    del list_of_positions[-x]
    x += 1

In [52]:
# Cette fonction permet d'ouvrir les différentes offres grâce au titre qui a été extrait avec la fonction ci-dessus. Ensuite elle extrait les informations pertinentes et en ajoute certaines comme le nom de la banque ainsi que le lien.
def recherche_info(position):
    driver.find_element_by_xpath('''//div[normalize-space(text()) = "{0}"]'''.format(position)).click()
    position_info = driver.find_element_by_xpath('''//*[@id="job"]/div[1]/div[1]''').text
    for mot in irrelevant_info:
        position_info = position_info.replace(mot,"")
    position_info = list(position_info.split('\n'))
    if len(position_info) == 6:
        del position_info[-1]
    del position_info[-3:]
    position_info.insert(2,"Natixis")
    position_info.insert(3,position)
    position_info.insert(4,driver.current_url)
    # position_info = pd.DataFrame([position_info], columns=["relative_id","date_added","source","title","link"])
    # print(position_info)
    # print(type(position_info))
    return position_info

In [53]:
# Création d'une liste qui accueillera les informations des nouvelles offres
session_data = []

# Recherche des informations dans la première page
for position in list_of_positions:
    position_info = recherche_info(position)
    if position_info[0] == last_csv_id:
        session_data = pd.DataFrame(session_data, columns=["relative_id","date_added","source","title","link"])
        natixis_data = pd.concat([session_data, last_csv])
        natixis_data = natixis_data.reset_index()
        del natixis_data["index"]
        natixis_data.to_csv("natixis.csv", mode='w+')
        natixis_data
        break
    session_data.append(position_info)
    driver.back()

In [54]:

# Rechercher des informations des pages suivantes jusqu'à la pénultième

page += 1
next_link = "https://recrutement.natixis.com/fr/jobs/list?country=FR&contractType=NA_INTERNSHIP&page={}".format(page)

while page < page_number:
    driver.get(next_link)
    texte = driver.find_element_by_id("job_list_results").text
    texte = texte.splitlines()
    list_of_positions = texte.copy()
    x = 1
    nb_max_result = (len(texte)/2)+1
    while x < nb_max_result:
        del list_of_positions[-x]
        x += 1

    for position in list_of_positions:
        position_info = recherche_info(position)
        if position_info[0] == last_csv_id:
            session_data = pd.DataFrame(session_data, columns=["relative_id","date_added","source","title","link"])
            natixis_data = pd.concat([session_data, last_csv])
            natixis_data = natixis_data.reset_index()
            del session_data["index"]
            natixis_data.to_csv("natixis.csv", mode='w+')
            natixis_data
            break
        session_data.append(position_info)
        driver.back()
    page +=1
    next_link = "https://recrutement.natixis.com/fr/jobs/list?country=FR&contractType=NA_INTERNSHIP&page={}".format(page)

#Ici on s'occupe de la dernière feuille
if page == page_number:
    driver.get(next_link)
    texte_integral_resultats_page_suivante = driver.find_element_by_id("job_list_results").text
    texte_integral_resultats_page_suivante = texte_integral_resultats_page_suivante.splitlines()
    titres_offres_page_suivante = texte_integral_resultats_page_suivante.copy()
    x = 1
    max = (len(texte_integral_resultats_page_suivante)/2)+1
    while x < max:
        del titres_offres_page_suivante[-x]
        x += 1
    for position in list_of_positions:
        position_info = recherche_info(position)
        if position_info[0] == last_csv_id:
            session_data = pd.DataFrame(session_data, columns=["relative_id","date_added","source","title","link"])
            natixis_data = pd.concat([session_data, last_csv])
            natixis_data = natixis_data.reset_index()
            del natixis_data["index"]
            natixis_data.to_csv("natixis.csv", mode='w+')
            natixis_data
            break
        session_data.append(position_info)
        driver.back()

C:\Users\josemanuel\AppData\Local\Temp\ipykernel_8848\94146985.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  session_data.append(position_info)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[normalize-space(text()) = "Stage - 6 mois - PPI Juriste Assurance - H/F"]"}
  (Session info: chrome=102.0.5005.115)
